context cache 만들

In [19]:
PROJECT_ID=!gcloud config get-value project
PROJECT_ID=PROJECT_ID[0]
LOCATION="us-east1"
MODEL_ID="gemini-1.5-pro-002"
DISPLAY_NAME="example-cache"

In [23]:
import vertexai
import datetime

from vertexai.generative_models import Part
from vertexai.preview import caching
from vertexai.preview.generative_models import GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [21]:
system_instruction = """
너는 전문 연구가이다.
너는 주워진 자원에서 항상 진실만을 말하며 새로운 사실을 만들지는 않는다.
제공된 논문을 기반으로 질문에 답변하자.
"""

contents = [
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = caching.CachedContent.create(
    model_name=MODEL_ID,
    system_instruction=system_instruction,
    contents=contents,
    ttl=datetime.timedelta(minutes=60),
    display_name=DISPLAY_NAME,
)

print(cached_content.name)

1295242289706172416


context cache 사

In [27]:
cached_content = caching.CachedContent(cached_content_name=cached_content.name)

model = GenerativeModel.from_cached_content(cached_content=cached_content)

response = model.generate_content("논문의 내용은 무엇인가?")

print(response.text)

이 보고서는 이미지, 오디오, 비디오 및 텍스트 이해를 포괄하는 뛰어난 기능을 가진 새로운 멀티모달 모델 제품군인 제미나이를 소개합니다. 제미나이 제품군은 복잡한 추론 작업에서 장치 메모리 제약 사용 사례에 이르기까지 애플리케이션에 적합한 울트라, 프로 및 나노 크기로 구성됩니다. 광범위한 벤치마크에 대한 평가 결과, 가장 기능이 뛰어난 제미나이 울트라 모델이 이러한 벤치마크 32개 중 30개에서 최첨단 기술을 발전시켰으며 특히 잘 연구된 시험 벤치마크 MMLU에서 인간 전문가 수준의 성능을 달성한 최초의 모델이자 조사한 20개의 멀티모달 벤치마크 각각에서 최첨단 기술을 개선한 모델임이 입증되었습니다. 교차 모달 추론 및 언어 이해에 대한 제미나이 제품군의 새로운 기능이 광범위한 사용 사례를 가능하게 할 것이라고 생각합니다. 제미나이, 제미나이 어드밴스드, 구글 AI 스튜디오 및 클라우드 버텍스 AI를 포함한 서비스를 통해 사용자에게 제미나이 모델을 책임감 있게 사후 교육하고 배포하는 접근 방식에 대해 논의합니다.


In [29]:
response = model.generate_content("What are the papers about?")

print(response.text)

이 논문은 이미지, 오디오, 비디오 및 텍스트 이해를 통해 놀라운 기능을 보이는 새로운 다중 모달 모델 제품군인 Gemini를 소개합니다. Gemini 제품군은 복잡한 추론 작업에서 장치 메모리 제약이 있는 사용 사례에 이르기까지 다양한 응용 프로그램에 적합한 Ultra, Pro 및 Nano 크기로 구성됩니다. 다양한 벤치마크에 대한 평가 결과, 가장 성능이 뛰어난 Gemini Ultra 모델은 32개 벤치마크 중 30개에서 최첨단 기술을 발전시켰으며, 특히 잘 연구된 시험 벤치마크 MMLU에서 인간 전문가 수준의 성능을 달성한 최초의 모델이었으며, 검토한 20개의 다중 모달 벤치마크 각각에서 최첨단 기술을 개선했습니다. 교차 모달 추론 및 언어 이해에 대한 Gemini 제품군의 새로운 기능은 다양한 사용 사례를 가능하게 할 것으로 예상됩니다. 이 보고서는 포스트 교육 접근 방식에 대해 논의하고 Gemini, Gemini Advanced, Google AI Studio 및 Cloud Vertex AI를 포함한 서비스를 통해 사용자에게 Gemini 모델을 책임감 있게 배포하는 방법에 대해 논의합니다.


context cache 정보 가져오기

In [30]:
cache_list = caching.CachedContent.list()

for cached_content in cache_list:
    print(f"Cache '{cached_content.name}' for model '{cached_content.model_name}'")
    print(f"Last updated at: {cached_content.update_time}")
    print(f"Expires at: {cached_content.expire_time}")

Cache '1295242289706172416' for model 'projects/younkicho-364103/locations/us-east1/publishers/google/models/gemini-1.5-pro-002'
Last updated at: 2025-01-17 05:07:55.869092+00:00
Expires at: 2025-01-17 06:07:55.845806+00:00
Cache '250407176156217344' for model 'projects/younkicho-364103/locations/us-east1/publishers/google/models/gemini-1.5-pro-002'
Last updated at: 2025-01-17 05:05:42.112630+00:00
Expires at: 2025-01-17 06:05:42.101691+00:00


context cache update

In [31]:
cached_content.update(ttl=datetime.timedelta(hours=1))
cached_content.refresh()

print(cached_content.expire_time)

2025-01-17 06:19:04.775970+00:00


context cache delete

In [32]:
cached_content.delete()

INFO:google.cloud.aiplatform.base:Deleting CachedContent : projects/429718924597/locations/us-east1/cachedContents/250407176156217344
